In [ ]:
import graphlab as gl

from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix

import pandas as pd
import numpy as np

In [ ]:
musicdata = "./dataset/amazon5core/digitalmusic/10ksample.csv"
healthdata = "./dataset/amazon5core/healthcare/10ksample.csv"

# loading data using gl
# musicsf = gl.SFrame.read_csv(musicdata, delimiter=",",
#                              usecols=["asin","reviewerID","overall","reviewText"],
#                              column_type_hints=[str, str, int, str])

healthsf = gl.SFrame.read_csv(healthdata, delimiter=",",
                              usecols=["asin","reviewerID","overall","reviewText"],
                              column_type_hints=[str, str, int, str])

In [ ]:
# divide the review and rating data
# ratings = musicsf[["asin", "reviewerID", "overall"]]
# reviews = musicsf[["asin", "reviewerID", "reviewText"]]


ratings = healthsf[["asin", "reviewerID", "overall"]]
reviews = healthsf[["asin", "reviewerID", "reviewText"]]

# train, test = gl.recommender.util.random_split_by_user(dataset=ratings,
#                                                        item_id="asin",
#                                                        user_id="reviewerID",
#                                                        max_num_users=100)



## cross validation structure
folds = gl.cross_validation.KFold(ratings, 10)
train, test = folds[3]

In [ ]:
## using recommender system lsa or svd


## random seed 22 is good.
shuffletrain = gl.cross_validation.shuffle(train, random_seed=22)

# for train, valid in folds:
model = gl.recommender.factorization_recommender.create(observation_data=train,
                                                        user_id="reviewerID",
                                                        item_id="asin",
                                                        target="overall",
                                                        num_factors=50, 
                                                        nmf=0, max_iterations=500,
                                                        solver='sgd',
                                                        verbose=1)
    
# print >> outF, model['coefficients']
# # view = model.views.overview(validation_set=)
# print >> outF, model.evaluate_precision_recall(dataset=valid)
# print >> outF, model.evaluate_rmse(valid, target='overall')
# print >> outF, "balibalibalibalibalibalibbalibbalibbalibalibali"

In [ ]:
out_rmse = model.evaluate_rmse(dataset=test, target='overall')
out_pr = model.evaluate_precision_recall(dataset=test, verbose=1)
print(out_rmse.viewkeys())
print(out_rmse['rmse_overall'])

In [ ]:
# python method list
methods = [method_name for method_name in dir(model) if callable(getattr(model, method_name))]
print(methods)

print(model.list_fields())

# print(out_pr['precision_recall_by_user'])
# print(out_pr['precision_recall_overall']['recall'].sum())
# model.show()

In [ ]:
model._get_queryable_methods()

In [ ]:
factors_asin = model['coefficients']['asin'][['factors', 'asin']]
factors_reviewerID = model['coefficients']['reviewerID'][['factors', 'reviewerID']]

In [ ]:
# factors_reviewerID
model.show()

In [ ]:
recs = model.recommend(users=["AFNCXMCX7VZWH"])

In [ ]:
## using topic modelling

# print(reviews)

one_item_reviews = []

for item in reviews['asin'].unique().sort():
    one_item_reviews.append([item, reviews.filter_by(item, 'asin')['reviewText']])

# print(reviews.filter_by('B00004OCLJ', 'asin')['reviewText'])


# lda = gl.topic_model.create(dataset=texttrain, validation_set=texttest, num_topics=100, num_iterations=200, verbose=1)



In [ ]:
one_item_reviews